# Data Modeling with Appache Cassandra
This project will do the following:
1. Set up a keyspace
2. Extact data from CSV files within a directory
3. Transform and load the data into three tables,
   which are designed upon final queries
   * The final queries are designed upon the following three questions:
     1. Give me the artist, song title and song's length in the music app history
        that was heard during  sessionId = 338, and itemInSession  = 4
        
     2. Give me only the following: name of artist, song (sorted by itemInSession)
        and user (first and last name) for userid = 10, sessionid = 182
        
     3. Give me every user name (first and last) in my music app history
        who listened to the song 'All Hands Against His Own'

### Import relevant Python packages 

In [ ]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

### Create a list of filepaths, from which to process CSV data files

In [ ]:
# Define filepath, in which the CSV files are located
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

# Get a list of filepaths
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

### Process the files to create a new CSV file, to be used for Apache Casssandra tables

In [ ]:
# create an empty list of rows
full_data_rows_list = [] 
    
# for every filepath in the filepath list:
for f in file_path_list:

# read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extract each data row one by one, and append it to the list
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below for total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below for entire rows
#print(full_data_rows_list)

# create a smaller event data csv file called event_datafile_new.csv that will be used to insert data into \
# the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

### Create a cluster

In [ ]:
# Connect to a local Cassandra instance (127.0.0.1)
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

In [ ]:
# Connect to a local Cassandra instance (127.0.0.1)
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

### Create keyspace

In [ ]:
session.execute('DROP KEYSPACE IF EXISTS sparkifyks')
session.execute('''CREATE KEYSPACE IF NOT EXISTS sparkifyks
                WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}''')

### Set keyspace

In [ ]:
session.execute('USE sparkifyks')
# or session.set_keyspace('sparkifyks')

### Create tables according to the following queries:
1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Create the first table, and run the first query

In [ ]:
# Create table
# Search conditions (sessionID and itemInSession) are used as primary key for speed
session.execute("DROP TABLE IF EXISTS info_session_music")
session.execute("""CREATE TABLE IF NOT EXISTS info_session_music
                (PRIMARY KEY (session_id, item_in_session),
                 session_id INT,
                 item_in_session INT,
                 artist TEXT,
                 song TEXT,
                 song_length FLOAT)""")

# ETL from the file into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO info_session_music (session_id, item_in_session, artist, song, song_length)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

# Run the query
rows = session.execute("""Select artist, song, song_length
                       FROM info_session_music
                       WHERE session_id = 338
                       AND item_in_session = 4""")

for row in rows:
    print('Artist:', row.artist, '\n',
          'Song:', row.song, '\n',
          'Song Length:', row.song_length, '\n')

### Create the second table, and run the second query

In [ ]:
# Create table
# Search conditions (user_id, session_id) are used as partition key for speed
session.execute("DROP TABLE IF EXISTS info_user_session_music")
session.execute("""CREATE TABLE IF NOT EXISTS info_user_session_music
                (PRIMARY KEY ((user_id, session_id), item_in_session),
                 user_id INT,
                 session_id INT,
                 item_in_session INT,
                 user_first_name TEXT,
                 user_last_name TEXT,
                 song TEXT,
                 artist TEXT)""")

# ETL from the file into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = "INSERT INTO info_user_session_music (user_id, session_id, item_in_session, \
                                                       user_first_name, user_last_name, song, artist)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[9], line[0]))

# Run the query
rows = session.execute("""Select artist, song, user_first_name, user_last_name
                       FROM info_user_session_music
                       WHERE user_id = 10
                       AND session_id = 182
                       ORDER BY item_in_session""")

for row in rows:
    print('Artist:', row.artist, '\n',
          'Song:', row.song, '\n',
          'User First Name:', row.user_first_name, '\n',
          'User Last Name:', row.user_last_name, '\n')

### Create the third table, and run the third query

In [ ]:
# Create table
# Search conditions (song and user_id) are used as primary key for speed
session.execute("DROP TABLE IF EXISTS info_song_user")
session.execute("""CREATE TABLE IF NOT EXISTS info_song_user
                (PRIMARY KEY (song, user_id),
                 song TEXT,
                 user_id INT,
                 user_first_name TEXT,
                 user_last_name TEXT)""")

# ETL from the file into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query3 = "INSERT INTO info_song_user (song, user_id, user_first_name, user_last_name)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))
        
# Run the query
rows = session.execute("""SELECT user_first_name, user_last_name, song
                       FROM info_song_user
                       WHERE song = 'All Hands Against His Own'""") 

for row in rows:
    print('User First Name:', row.user_first_name, '\n',
          'User Last Name:', row.user_last_name, '\n',
          'Song:', row.song, '\n')

### Drop the tables before closing the session

In [ ]:
session.execute('DROP TABLE info_session_music')
session.execute('DROP TABLE info_user_session_music')
session.execute('DROP TABLE info_song_user')

### Close the session and the cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()